In [1]:
from os.path import isfile, join
from os import listdir
import re
import requests
import pandas as pd

In [2]:
OUTPUT_PATH = join('..', 'data')

In [3]:
url = "https://eur-lex.europa.eu/legal-content/EN/TXT/HTML/?uri=CELEX:32015R2450&from=EN"
# amendment 2016
#url = "https://eur-lex.europa.eu/legal-content/EN/TXT/HTML/?uri=CELEX:32016R1868&from=EN"
# amendment 2017
#url = "https://eur-lex.europa.eu/legal-content/EN/TXT/HTML/?uri=CELEX:32017R2189&from=EN"
# amendment 2018
#url = "https://eur-lex.europa.eu/legal-content/EN/TXT/HTML/?uri=CELEX:32018R1844&from=EN"
# amendment 2019
#url = "https://eur-lex.europa.eu/legal-content/EN/TXT/HTML/?uri=CELEX:32019R2103&from=EN"
filename = "log.html"

In [4]:
r = requests.get(url)
f = open(join(OUTPUT_PATH, filename),'wb+')
f.write(r.content)
f.close()

In [5]:
f = open(join(OUTPUT_PATH, filename),'rb')
text = f.read()
f.close()

In [6]:
from bs4 import BeautifulSoup, NavigableString
soup = BeautifulSoup(text, 'html.parser')

In [29]:
def between(cur, end):
    while cur and cur != end:
        if isinstance(cur, NavigableString):
            text = cur.strip()
            if len(text):
                yield text
        cur = cur.next_element

df = pd.DataFrame()
tables = soup.find_all("table")
for table in tables:
    trs = table.find_all("tr", attrs={'class', 'table'})
    datapoints = []
    for tr in trs:
        tds = tr.find_all("td", attrs={'class', 'table'})
        items = []
        for td in tds:
            items.append(td.get_text().replace("\n", ""))
        datapoints.append(items)
    if datapoints:
        if datapoints[0] == ['\xa0', 'ITEM', 'INSTRUCTIONS']:
            sibling = table.find_previous_sibling('p', {'class':'ti-grseq-1'})
            template_name = (sibling.get_text().replace("\n", ""))
            b = list(between(sibling, table))
            template_text = '\n\n'.join(text for text in b[1:])
            if "—" in template_name:
                template_name = template_name.split("—")
            elif "–" in template_name:
                template_name = template_name.split("–")
            short_name = template_name[0].strip()
            if len(template_name) >= 2:
                long_name = ",".join(template_name[1:]).strip()
            else:
                long_name = ""
            for datapoint in datapoints[1:]:
                df = df.append(pd.DataFrame([[short_name, long_name] + [template_text] + datapoint]), ignore_index = True)
df.loc[:, 3] = df.loc[:, 3].str.lower()
df.loc[:, 3] = df.loc[:, 3].str.replace("/", "")

In [30]:
df[0].unique()

array(['S.01.01', 'S.01.02', 'S.01.03', 'S.02.01', 'S.02.02', 'S.03.01',
       'S.03.02', 'S.03.03', 'S.04.01', 'S.04.02', 'S.05.01', 'S.05.02',
       'S.06.01', 'S.06.02', 'S.06.03', 'S.07.01', 'S.08.01', 'S.08.02',
       'S.09.01', 'S.10.01', 'S.11.01', 'S.12.01', 'S.12.02', 'S.13.01',
       'S.14.01', 'S.15.01', 'S.15.02', 'S.16.01.', 'S.17.01', 'S.17.02',
       'S.18.01', 'S.19.01', 'S.20.01', 'S.21.02', 'S.22.01', 'S.22.02',
       'S.22.03', 'S.22.04', 'S.22.05', 'S.22.06', 'S.23.01', 'S.23.02',
       'S.23.03', 'S.23.04', 'S.24.01', 'S.25.01', 'S.25.02', 'S.25.03',
       'S.26.01', 'S.26.02', 'S.26.03', 'S.26.04', 'S.26.05', 'S.26.06',
       'S.26.07', 'S.27.01', 'S.28.01', 'S.28.02', 'S.29.01', 'S.29.02',
       'S.29.03', 'S.29.04', 'S.30.01', 'S.30.02', 'S.30.03', 'S.30.04',
       'S.31.01', 'S.31.02', 'S.36.01', 'S.36.02', 'S.36.03', 'S.36.04',
       'S.32.01', 'S.33.01', 'S.34.01', 'S.35.01', 'S.37.01'],
      dtype=object)

In [31]:
import pandas as pd
import re
import os
from os import listdir, walk, makedirs
from os.path import isfile, join, exists, basename
from io import StringIO

In [32]:
DOCS_PATH ="..\\docs\\"
DATA_PATH ="..\\data\\"
RULES_PATH = "..\\Solvency2-rules\\"

DIR_NAME = "instructions"

In [33]:
templates = df[0].unique()

if not exists(join(DOCS_PATH, DIR_NAME)):
    makedirs(join(DOCS_PATH, DIR_NAME))

In [34]:
df.head(5)

,0,1,2,3,4,5
0,S.01.01,Content of the submission,General comments:\n\nThis section relates to o...,z0010,Ring–fenced fund/matching portfolio/remaining ...,Identifies whether the reported figures are wi...
1,S.01.01,Content of the submission,General comments:\n\nThis section relates to o...,z0020,Fund/Portfolio number,"When item Z0010 = 1, identification number for..."
2,S.01.01,Content of the submission,General comments:\n\nThis section relates to o...,c0010r0010,S.01.02 — Basic Information — General,This template shall always be reported. The on...
3,S.01.01,Content of the submission,General comments:\n\nThis section relates to o...,c0010r0020,S.01.03 — Basic Information — RFF and matching...,One of the options in the following closed lis...
4,S.01.01,Content of the submission,General comments:\n\nThis section relates to o...,c0010r0030,S.02.01 — Balance sheet,One of the options in the following closed lis...


In [35]:
def write_file(filename, string):
    file = open(filename, "w", encoding = 'utf8')
    file.write(string.getvalue())
    file.close()

def header(io, text):
    io.write("=" * len(text) + "\n")
    io.write(text + "\n")
    io.write("=" * len(text) + "\n\n")
    return io

def section(io, text):
    io.write(text + "\n")
    io.write("-" * len(text) + "\n\n")
    return io


In [36]:
templates = df[0].unique()

for template in templates:
    string = StringIO()
    string = header(string, template + " - " + df[df[0]==template][1].values[0])
    string = section(string, "Instructions")
    string.write("\n")
    string.write(df[df[0]==template][2].values[0])
    string.write("\n\n\n")
    for row in df[df[0]==template].index:
        string = section(string, df.loc[row, 3] + " - " + df.loc[row, 4])
        string.write("\n")
        string.write(df.loc[row, 5])
        string.write("\n\n\n")
    write_file(join(DOCS_PATH, DIR_NAME, template.replace(".", "_") + ".rst"), string)
    string.close()

In [4]:
# amendment 2016
url = "https://eur-lex.europa.eu/legal-content/EN/TXT/HTML/?uri=CELEX:32016R1868&from=EN"
filename = "amendment_2016.html"

In [5]:
r = requests.get(url)
f = open(join(OUTPUT_PATH, filename),'wb+')
f.write(r.content)
f.close()

In [6]:
f = open(join(OUTPUT_PATH, filename),'rb')
text = f.read()
f.close()

In [7]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(text, 'html.parser')

In [75]:
tables = []
df = pd.DataFrame()
annex_2 = soup.find("div", attrs = {'id': 'L_2016286EN.01004501'})
items = annex_2.find_all("tr")
for item in items:
    tables.append(item)

In [80]:
print(tables[15].prettify())

<tr class="table">
 <td class="table" valign="top">
  <p class="tbl-txt">
   ‘R0290/C0020
  </p>
 </td>
 <td class="table" valign="top">
  <p class="tbl-txt">
   Initial absolute values before shock — Assets — Equity risk — qualifying infrastructure equities
  </p>
 </td>
 <td class="table" valign="top">
  <p class="tbl-txt">
   This is the initial absolute value of the assets sensitive to the equity risk for qualifying infrastructure equities.
  </p>
  <p class="tbl-txt">
   Recoverables from reinsurance and SPVs shall not be included in this cell.
  </p>
 </td>
</tr>

